<h1> <center> Table of Contents </center> </h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#1.-Required-Libraries"> Required Libraries</a></li>
        <li><a href="#2.-Importing-the-Data"> Importing the Data</a></li>
        <li><a href="#3.-Creating-Corpus"> Creating-Corpus</a> </li>
        <li><a href="#4.-Using-Fuzzywuzzy-to-get-best-matched-dashboards"> Using Fuzzywuzzy to get best matched dashboards</a></li> 
      </li>
    </ol>
</div>

# 1. Required Libraries 

## pip installs

In [1]:
#!pip install python-Levenshtein
#!pip install "fuzzywuzzy==0.18.0"
#!pip install rapidfuzz
#!pip install spacy
#!pip install gensim
#!pip install rank_bm25
#!python -m spacy download en_core_web_lg
#!pip install fast-autocomplete

## Importing Libraries

In [2]:
import pandas as pd
from pandas.core.common import flatten
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import rapidfuzz
from rapidfuzz import process, utils

import timeit
import time

import warnings
warnings.filterwarnings('ignore')

/Users/rasheshkothari/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## 2. Importing the Data

In [3]:
# importing the files
dfdashboards = pd.read_csv('microstrategy_and_dashboard.csv')
dfmetrics = pd.read_csv('tableau_metric.csv')
dfuserquey = pd.read_csv('sample_user_searchs.csv')

In [4]:
def preprocess_datafiles(dashboard , metric):

    # converting column values to lowercase
    dashboard = dashboard.apply(lambda x: x.astype(str).str.lower())
    metric = metric.apply(lambda x: x.astype(str).str.lower())
    
    # splitting caption and descriptors columns 
    dashboard = dashboard[['dashboards','caption','Global_usage']]
    dashboard['metric_names'] = dashboard.caption.str.split('|')
    metric['descriptors'] = metric.descriptors.str.split('|')
    
    # converting metric names and descriptors in each row values to multiple rows
    temp1 = dashboard.set_index(['dashboards'])['metric_names'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'caption'})
    temp2 = metric.set_index(['metric_name'])['descriptors'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'descriptor'})
    
    # merging metrics and descriptors
    master = temp1.merge(temp2, how = 'inner', left_on ='caption', right_on = 'metric_name')[['dashboards','metric_name','descriptor']]
    master = master.rename(columns={"dashboards":"dashboard_names", "metric_name": "metric_names", "descriptor": "descriptor_names"})

    return master



In [5]:
# Final Dataframe
master = preprocess_datafiles(dfdashboards,dfmetrics)
master.head()

,dashboard_names,metric_names,descriptor_names
0,ad analysis,beer style level value,macro style value
1,ad analysis,beer style level value,mezzo style value
2,ad analysis,beer style level value,micro style valu
3,ad analysis,beer style level value,beer style value
4,ad analysis,beer style level value,altbier


In [6]:
master[master['descriptor_names'] == 'macro style value']

,dashboard_names,metric_names,descriptor_names
0,ad analysis,beer style level value,macro style value
85,ad recap,beer style level value,macro style value
170,aggregate sales per pt vs. cwd,beer style level value,macro style value
255,brands on ad,beer style level value,macro style value
340,brewery comparisons,beer style level value,macro style value
...,...,...,...
3306,sales per pt vs. cwd,beer style levels,macro style value
3311,style analysis,beer style levels,macro style value
3316,topline,beer style levels,macro style value
3321,velocity,beer style levels,macro style value


## 3. Creating Corpus

In [7]:
dashboard_names_list = []
metric_names_list = []
descriptor_names_list = []

def final_corpus(masterDf):
    
    dashboard_names_list = list(masterDf.dashboard_names.unique())
    metric_names_list = list(masterDf.metric_names.unique())
    descriptor_names_list = list(masterDf.descriptor_names.unique())
    
    mastercorpus = dashboard_names_list + metric_names_list + descriptor_names_list
    mastercorpus = list(filter(None,mastercorpus))
    
    return mastercorpus

mastercorpus = final_corpus(master)
mastercorpus[:5]

['ad analysis',
 'ad recap',
 'aggregate sales per pt vs. cwd',
 'brands on ad',
 'brewery comparisons']

## 4. Using Fuzzywuzzy to get best matched dashboards

FuzzyWuzzy is a library of Python which is used for string matching. Fuzzy string matching is the process of finding strings that match a given pattern. Basically it uses <b>Levenshtein Distance</b> to calculate the differences between sequences.


In [17]:
# Logic to get the dashboards

dashboard_final ={}

def dashboard_names_suggestion(master, suggestions):
    
    dashboard_names = []   
    
    for i in suggestions:
        
        if i in dashboard_names_list:    
            dashboard_names.append(i)
        
        elif i in metric_names_list:    
            dashboard_names = dashboard_names + (master.loc[(master.metric_names.str.lower() == i)].dashboard_names).to_list()
        
        else:
            dashboard_names = dashboard_names + (master.loc[(master.descriptor_names.str.lower() == i)].dashboard_names).to_list()

    return dashboard_names

In [18]:
token_set_ratio_sugg = {}

def fuzzywuzzy_scorers_suggestions(usersearch, corpus):
    
    for token in corpus:
        sugg_score = fuzz.token_set_ratio(usersearch, token)
        token_set_ratio_sugg[token] = sugg_score

        
def fuzzywuzzy_scorers_similiarity(scorers,suggestions_count,sort_scorers):
   
    scoreDf = pd.DataFrame()
    scoreDf = scoreDf.from_dict([token_set_ratio_sugg]).T.reset_index()
    scoreDf.columns = ['suggestion','token_set_ratio_sugg']
#     scoreDf['mean'] = scoreDf.mean(axis = 1)
    
    return scoreDf[scorers].sort_values(by = sort_scorers, ascending = False).head(suggestions_count)

In [35]:
input_query = 'Macro style value amber ale 5.5 percent'

In [36]:
# creating column names, list to append suggestions

fuzzyScorers = ['suggestion','token_set_ratio_sugg']
scorersSorting = ['token_set_ratio_sugg']

allScorersOutput = []

fuzzywuzzySearchOutput = []


# FUNCTION CALLING 
fuzzywuzzy_scorers_suggestions(input_query.lower(), mastercorpus)

# FUNCTION CALLING 
scoreSortedDf = fuzzywuzzy_scorers_similiarity(scorers=fuzzyScorers, suggestions_count = 10,sort_scorers= scorersSorting)



# FUNCTION CALLING
fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(master, scoreSortedDf['suggestion'][:10]))

fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
allScorersOutput.append(fuzzywuzzySearchOutput[:10])


column_names = ['Dashboard Suggestions']
finalSuggestionDF = pd.DataFrame(allScorersOutput).transpose().set_axis(column_names, axis =1)

print('User Search : ' + str(input_query))

print("Displaying the dashboards names by each scorer methods.")
display(finalSuggestionDF)

User Search : Macro style value amber ale 5.5 percent
Displaying the dashboards names by each scorer methods.


,Dashboard Suggestions
0,ad analysis
1,ad recap
2,aggregate sales per pt vs. cwd
3,brands on ad
4,brewery comparisons
5,company
6,company brands over time
7,r geography
8,r geography over time
9,company


In [37]:
scoreSortedDf['suggestion'][:10].to_list()

['macro style value',
 '5.5 percent',
 'amber ale',
 '5 percent',
 '5',
 '6.5 percent',
 '5.3 percent',
 '3.5 percent',
 '8.5 percent',
 '9.5 percent']

In [38]:
display(fuzzywuzzyDashboardsList[:10])

['ad analysis',
 'ad recap',
 'aggregate sales per pt vs. cwd',
 'brands on ad',
 'brewery comparisons',
 'company ',
 'company brands over time',
 'r geography',
 'r geography over time',
 'company']

In [39]:
display(master[master['descriptor_names'] == '5.5 percent'])
display(master[master['descriptor_names'] == 'micro style value'][:10])
display(master[master['descriptor_names'] == 'amber ale'][:10])

,dashboard_names,metric_names,descriptor_names
23809,ad analysis,alcoholic vs. non-alcoholic,5.5 percent
23895,aggregate sales per pt vs. cwd,alcoholic vs. non-alcoholic,5.5 percent
23981,merch vs. non-merch,alcoholic vs. non-alcoholic,5.5 percent


,dashboard_names,metric_names,descriptor_names
3148,ad analysis,beer style levels,micro style value
3153,ad recap,beer style levels,micro style value
3158,aggregate sales per pt vs. cwd,beer style levels,micro style value
3163,brands on ad,beer style levels,micro style value
3168,brewery comparisons,beer style levels,micro style value
3173,company,beer style levels,micro style value
3178,company brands over time,beer style levels,micro style value
3183,r geography,beer style levels,micro style value
3188,r geography over time,beer style levels,micro style value
3193,company,beer style levels,micro style value


,dashboard_names,metric_names,descriptor_names
3330,ad analysis,beer style,amber ale
3357,ad recap,beer style,amber ale
3384,aggregate sales per pt vs. cwd,beer style,amber ale
3411,brands on ad,beer style,amber ale
3438,brewery comparisons,beer style,amber ale
3465,company,beer style,amber ale
3492,company brands over time,beer style,amber ale
3519,r geography,beer style,amber ale
3546,r geography over time,beer style,amber ale
3573,company,beer style,amber ale


In [40]:
# metric_suggestion_list = ['amber ale','dollar sales','dollar sales per pt']
metric_suggestion_list = ['seasonal']

master.loc[master.descriptor_names.isin(metric_suggestion_list)]
df1 = master[master['descriptor_names'] == metric_suggestion_list[0]]
df2 = master[master['descriptor_names'] == metric_suggestion_list[1]]
df3 = master[master['descriptor_names'] == metric_suggestion_list[2]]

dfs = [df1, df2, df3]

from functools import partial, reduce

df_merged= reduce(lambda left,right: pd.merge(left,right,on='dashboard_names'), dfs)
df_merged

IndexError: list index out of range

In [ ]:
metric_suggestion_list = ['6','the fighting 69th regiment','fighting 69th regiment']
# master.loc[master.descriptor_names.isin(metric_suggestion_list)]
df1 = master[master['descriptor_names'] == metric_suggestion_list[0]]
df2 = master[master['descriptor_names'] == metric_suggestion_list[1]]
df3 = master[master['descriptor_names'] == metric_suggestion_list[2]]

dfs = [df1, df2, df3]

from functools import partial, reduce

df_merged= reduce(lambda left,right: pd.merge(left,right,on='dashboard_names'), dfs)
df_merged['dashboard_names'].unique()

In [28]:
# check each metric in corpus and associated dashboard 

In [29]:
master[master['descriptor_names'] == 'amber ale']

,dashboard_names,metric_names,descriptor_names
3330,ad analysis,beer style,amber ale
3357,ad recap,beer style,amber ale
3384,aggregate sales per pt vs. cwd,beer style,amber ale
3411,brands on ad,beer style,amber ale
3438,brewery comparisons,beer style,amber ale
3465,company,beer style,amber ale
3492,company brands over time,beer style,amber ale
3519,r geography,beer style,amber ale
3546,r geography over time,beer style,amber ale
3573,company,beer style,amber ale


In [34]:
pd.concat([finalSuggestionDF['Dashboard Suggestions'],scoreSortedDf['suggestion'].reset_index()], axis =1)[['Dashboard Suggestions','suggestion']]

,Dashboard Suggestions,suggestion
0,ad analysis,5.5 percent
1,aggregate sales per pt vs. cwd,5
2,merch vs. non-merch,amber ale
3,brands on ad,5 percent
4,brewery comparisons,5.4 percent
5,company,7.5 percent
6,company brands over time,8.5 percent
7,r geography,5.9 percent
8,r geography over time,5.8 percent
9,company,5.7 percent


In [31]:
pd.concat([finalSuggestionDF['Dashboard Suggestions'],scoreSortedDf['suggestion'].reset_index()], axis =1)[['Dashboard Suggestions']]

,Dashboard Suggestions
0,ad analysis
1,aggregate sales per pt vs. cwd
2,merch vs. non-merch
3,brands on ad
4,brewery comparisons
5,company
6,company brands over time
7,r geography
8,r geography over time
9,company
